In [1]:
import psycopg2
import pandas as pd

from dotenv import load_dotenv
import os
load_dotenv()

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
conn = psycopg2.connect(**db_config)

sql_fulfillment = "SELECT * FROM ml_fulfillment3"

df_fulfillment = pd.read_sql(sql_fulfillment, conn)

df_fulfillment.head(1)

C:\Users\couti\AppData\Local\Temp\ipykernel_15920\3359825672.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fulfillment = pd.read_sql(sql_fulfillment, conn)


,ml_inventory_id,ml_item_id,variation_id,nad_status,nad_quantity,total,available_quantity,not_available_quantity,type,created_at
0,JFGN34621,MLB924922735,nan,NaN,0,71,71,0,item,2023-11-07 16:31:05.045825


In [4]:
df_fulfillment.columns

Index(['ml_inventory_id', 'ml_item_id', 'variation_id', 'nad_status',
       'nad_quantity', 'total', 'available_quantity', 'not_available_quantity',
       'type', 'created_at'],
      dtype='object')

In [5]:
cols = ['ml_inventory_id','ml_item_id','variation_id']
df_fulfillment_new = df_fulfillment[cols]
df_fulfillment_new

,ml_inventory_id,ml_item_id,variation_id
0,JFGN34621,MLB924922735,nan
1,ERIM51807,MLB949788598,nan
2,HVCS52763,MLB950214971,nan
3,ZQIV51979,MLB950295475,nan
4,RBUB53192,MLB950295955,nan
...,...,...,...
816,PPWL59058,MLB2986538160,175887797708.0
817,RXXA58942,MLB2986538160,175887797707.0
818,POTW78210,MLB3106901850,nan
819,POTW78210,MLB3106901850,nan


In [10]:
from datetime import datetime, timedelta

today = datetime.now()

date_list = [today - timedelta(days=i) for i in range(30)]

df_dates = pd.DataFrame({'created_at': date_list})

df_dates

,created_at
0,2023-11-07 15:30:03.599530
1,2023-11-06 15:30:03.599530
2,2023-11-05 15:30:03.599530
3,2023-11-04 15:30:03.599530
4,2023-11-03 15:30:03.599530
5,2023-11-02 15:30:03.599530
6,2023-11-01 15:30:03.599530
7,2023-10-31 15:30:03.599530
8,2023-10-30 15:30:03.599530
9,2023-10-29 15:30:03.599530


In [11]:
dfs = []

for index, row in df_fulfillment_new.iterrows():
    df_temp = pd.concat([pd.DataFrame(row).T] * 30, ignore_index=True)
    df_temp['created_at'] = df_dates['created_at']
    dfs.append(df_temp)

df_final = pd.concat(dfs, ignore_index=True)

df_final.shape

(24630, 4)

In [12]:
821*30

24630

In [13]:
# Crie um dicionário com os valores padrão para cada coluna
default_values = {
    'nad_status': 'NaN',
    'nad_quantity': 0,
    'total': 1,
    'available_quantity': 1,
    'not_available_quantity': 0,
    'type': 'default_type'
}

# Use o método assign para adicionar as colunas com valores padrão
df_final = df_final.assign(**default_values)

# Agora df_final conterá as colunas adicionadas com os valores padrão
df_final

,ml_inventory_id,ml_item_id,variation_id,created_at,nad_status,nad_quantity,total,available_quantity,not_available_quantity,type
0,JFGN34621,MLB924922735,nan,2023-11-07 15:30:03.599530,NaN,0,1,1,0,default_type
1,JFGN34621,MLB924922735,nan,2023-11-06 15:30:03.599530,NaN,0,1,1,0,default_type
2,JFGN34621,MLB924922735,nan,2023-11-05 15:30:03.599530,NaN,0,1,1,0,default_type
3,JFGN34621,MLB924922735,nan,2023-11-04 15:30:03.599530,NaN,0,1,1,0,default_type
4,JFGN34621,MLB924922735,nan,2023-11-03 15:30:03.599530,NaN,0,1,1,0,default_type
...,...,...,...,...,...,...,...,...,...,...
24625,UCQQ19395,MLB3288695231,nan,2023-10-13 15:30:03.599530,NaN,0,1,1,0,default_type
24626,UCQQ19395,MLB3288695231,nan,2023-10-12 15:30:03.599530,NaN,0,1,1,0,default_type
24627,UCQQ19395,MLB3288695231,nan,2023-10-11 15:30:03.599530,NaN,0,1,1,0,default_type
24628,UCQQ19395,MLB3288695231,nan,2023-10-10 15:30:03.599530,NaN,0,1,1,0,default_type


In [14]:
df_fulfillment.head(10)

,ml_inventory_id,ml_item_id,variation_id,nad_status,nad_quantity,total,available_quantity,not_available_quantity,type,created_at
0,JFGN34621,MLB924922735,nan,NaN,0,71,71,0,item,2023-11-07 16:31:05.045825
1,ERIM51807,MLB949788598,nan,transfer,3,36,33,3,item,2023-11-07 16:31:05.045825
2,HVCS52763,MLB950214971,nan,transfer,2,3,1,2,item,2023-11-07 16:31:05.045825
3,ZQIV51979,MLB950295475,nan,NaN,0,6,6,0,item,2023-11-07 16:31:05.045825
4,RBUB53192,MLB950295955,nan,transfer,3,21,18,3,item,2023-11-07 16:31:05.045825
5,FXMN50419,MLB950296037,nan,NaN,0,0,0,0,item,2023-11-07 16:31:05.045825
6,TDFV51283,MLB950297972,nan,NaN,0,0,0,0,item,2023-11-07 16:31:05.045825
7,PKNN52962,MLB950303724,nan,NaN,0,3,3,0,item,2023-11-07 16:31:05.045825
8,MAEG54107,MLB950338822,nan,NaN,0,16,16,0,item,2023-11-07 16:31:05.045825
9,RYWV54124,MLB950341824,nan,NaN,0,0,0,0,item,2023-11-07 16:31:05.045825
